# Creating a basic chat experience with context variables

In this example, we show how you can build a simple chat bot by sending and updating context with your requests. 

We introduce the Context Variables object which in this demo functions similarly as a key-value store that you can use when running the kernel.

The context is local (i.e. in your computer's RAM) and not persisted anywhere beyond the life of this Jupyter session.

In future examples, we will show how to persist the context on disk so that you can bring it into your applications.  

In this chat scenario, as the user talks back and forth with the bot, the context gets populated with the history of the conversation. During each new run of the kernel, the context can provide the AI with its variables' content. 

In [1]:
#r "nuget: Microsoft.SemanticKernel, 0.8.48.1-preview"
#!import config/Settings.cs


using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Configuration;
using Microsoft.SemanticKernel.SemanticFunctions;
using Microsoft.SemanticKernel.KernelExtensions;
using Microsoft.SemanticKernel.Orchestration;

IKernel kernel = Microsoft.SemanticKernel.Kernel.Builder.Build();

// Configure AI backend used by the kernel
var (useAzureOpenAI, model, azureEndpoint, apiKey, orgId) = Settings.LoadFromFile();
if (useAzureOpenAI)
    kernel.Config.AddAzureOpenAICompletionBackend("davinci", model, azureEndpoint, apiKey);
else
    kernel.Config.AddOpenAICompletionBackend("davinci", model, apiKey, orgId);

Installed Packages Microsoft.SemanticKernel, 0.8.48.1-preview

Let's define a prompt outlining a dialogue chat bot.

In [2]:
const string skPrompt = @"
ChatBot can have a conversation with you about any topic.
It can give explicit instructions or say 'I don't know' if it does not have an answer.

{{$history}}
Human: {{$human_input}}
ChatBot:";

var promptConfig = new PromptTemplateConfig
{
    Completion =
    {
        MaxTokens = 2000,
        Temperature = 0.7,
        TopP = 0.5,
    }
};

Register your semantic function

In [3]:
var promptTemplate = new PromptTemplate(skPrompt, promptConfig, kernel);
var functionConfig = new SemanticFunctionConfig(promptConfig, promptTemplate);
var chatFunction = kernel.RegisterSemanticFunction("ChatBot", "Chat", functionConfig);

Initialize your context

In [4]:
var context = new ContextVariables();

var history = "";
context.Set("history", history);

Chat with the Bot

In [5]:
var human_input = "Hi, I'm looking for a specific book suggestion.";
context.Set("human_input", human_input);

var bot_answer = await kernel.RunAsync(context, chatFunction);

Update the history with the output

In [6]:
history += $"\nHuman: {human_input}\nMelody: {bot_answer}\n";
context.Update(history);

Console.WriteLine(context);


Human: Hi, I'm looking for a specific book suggestion.
Melody:  Hello, I'm happy to help you find a book. What kind of book are you interested in?



Keep Chatting!

In [7]:
Func<string, Task> Chat = async (string input) => {
    // Save new message in the context variables
    context.Set("human_input", input);

    // Process the user message and get an answer
    var answer = await kernel.RunAsync(context, chatFunction);

    // Append the new interaction to the chat history
    history += $"\nHuman: {input}\nMelody: {answer}\n"; context.Set("history", history);
    
    // Show the response
    Console.WriteLine(context);
};

In [12]:
await Chat("I would like a non-fiction book suggestion about Greece history. Please only list one book.");

  Alright, I will only list one book for you. How about this one:

- Greece: A History by Alexander Eliot: This is a comprehensive and engaging history of Greece, written by the American historian and writer Alexander Eliot. It covers the origins, achievements, and legacy of the ancient Greeks, as well as the modern history and politics of the country. It is a well-researched and accessible book, with illustrations, maps, and photographs.


In [14]:
await Chat("that sounds interesting, what are some of the topics I will learn about?");

 Some of the topics you will learn about are:

- The Minoan and Mycenaean civilizations, the first advanced cultures of Greece
- The rise and fall of the city-states, such as Athens, Sparta, Thebes, and Corinth
- The golden age of Greece, marked by the achievements of Pericles, Socrates, Plato, Aristotle, and Alexander the Great
- The Hellenistic period, when Greek culture spread across the Mediterranean and Asia
- The Roman and Byzantine domination of Greece, and the influence of Christianity and Islam
- The Ottoman occupation of Greece, and the struggle for independence and democracy
- The modern history of Greece, including the World Wars, the Civil War, the dictatorship, and the European integration


In [15]:
await Chat("Which topic from the ones you listed do you think most people find interesting?");

  That is a hard question to answer, as different people may have different preferences and opinions. However, I think that some of the topics that most people find interesting are:

- The golden age of Greece, as it showcases the remarkable achievements and contributions of the ancient Greeks to various fields of knowledge and culture
- The Hellenistic period, as it reveals the diversity and complexity of the Greek world, and its interactions with other civilizations and cultures
- The modern history of Greece, as it relates to the current issues and challenges that the country faces, and its role in the European and global context


In [16]:
await Chat("could you list some more books I could read about the topic(s) you mentioned?");

 Sure, I can list some more books that you could read about the topics I mentioned. Here are some examples:

- The Parthenon by Mary Beard: This is a fascinating and informative book, written by the British classicist and historian Mary Beard. It explores the history, meaning, and significance of the Parthenon, the iconic temple of Athens, and its impact on the world's imagination and culture.
- Alexander the Great by Philip Freeman: This is a captivating and authoritative biography, written by the American historian and writer Philip Freeman. It chronicles the life, achievements, and legacy of Alexander the Great, the king of Macedonia, who conquered most of the known world and created a vast empire.
- The Balkans: A Short History by Mark Mazower: This is a concise and insightful history, written by the British historian and professor Mark Mazower. It covers the turbulent and complex history of the Balkans, the region that includes Greece and its neighboring countries, and its role in